In [22]:
!pip install lightgbm
!pip install hypopt --upgrade
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import joblib
from joblib import dump, load
import hypopt
from hypopt import GridSearch
import pickle
from tensorflow import keras
import kerastuner as kt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

You should consider upgrading via the 'c:\users\vince\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\vince\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
X_all = pd.read_pickle("X_all.pkl")
Y_all = pd.read_pickle("Y_all.pkl")

In [ ]:
X_all = np.asarray(X_all)
Y_all = np.asarray(Y_all)

In [14]:
np.save('X_all.npy', X_all)
np.save('Y_all.npy', Y_all)

In [10]:
samples_classifications = dict()

<h3>Learning single models</h3> (Subject Independent)

KNN tuning and cross validation

In [41]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])


for index, (train, test) in enumerate(skf.split(X_all, Y_all)):
    print('Iteration ', index)
    
    # Creating folds and validation set for tuning
    X_train = X_all[train]
    y_train = Y_all[train]
    X_test = X_all[test]
    y_test = Y_all[test]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    
    # Defining search space
    param_grid = { 
                    'n_neighbors' : [1,3,5,7,9,11,13,15], #[x for x in range(1, 105, 2)],
                    'weights' : ['uniform', 'distance'],
                    'metric' : ['minkowski','euclidean', 'manhattan'],
                    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
                    'leaf_size' : [10, 20, 30, 40 , 50, 60, 70, 80, 90]
                    
        
    }
    
    # Tuning on defined validation set of current iteration
    gs = GridSearch(model = KNeighborsClassifier(), param_grid = param_grid, parallelize=False)
    gs.fit(X_train, y_train, X_val, y_val, scoring='f1')
    
    dump(gs.best_estimator_, './models/knn_'+str(index)+'.joblib') 
    preds = gs.best_estimator_.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    
    for i,x in enumerate(test):
        if x in samples_classifications:
            samples_classifications[x]['KNN']= preds[i]
        else:
            samples_classifications[x]= {'KNN': preds[i]}
    
    print(confusion_matrix(y_test, preds))
    print('f1-score', gs.best_estimator_.score(X_test, y_test))
    print(gs.best_params)
    print('\n')

Iteration  0
[[135  48]
 [ 65 144]]
f1-score 0.7117346938775511
{'algorithm': 'ball_tree', 'leaf_size': 30, 'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'}


Iteration  1
[[146  37]
 [ 77 132]]
f1-score 0.7091836734693877
{'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}


Iteration  2
[[149  34]
 [ 70 139]]
f1-score 0.7346938775510204
{'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}


Iteration  3
[[129  54]
 [ 71 138]]
f1-score 0.6811224489795918
{'algorithm': 'brute', 'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'distance'}


Iteration  4
[[137  46]
 [ 72 137]]
f1-score 0.6989795918367347
{'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}


Iteration  5
[[129  54]
 [ 80 129]]
f1-score 0.6581632653061225
{'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 1, 'weights'

In [43]:
final_cm

array([[1358,  471],
       [ 757, 1331]])

ET tuning and cross validation

In [14]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])


for index, (train, test) in enumerate(skf.split(X_all, Y_all)):
    print('Iteration ', index)
    
    # Creating folds and validation set for tuning
    X_train = X_all[train]
    y_train = Y_all[train]
    X_test = X_all[test]
    y_test = Y_all[test]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    
    # Defining search space
    param_grid = {
                    'n_estimators': range(10,200,10),
                    'max_depth': [2,8,16,32,50],
                    'min_samples_split': [2,4,6],
                    'min_samples_leaf': [1,2],
                    'max_features': ['auto','sqrt','log2'],    
                    'bootstrap': [True, False]
    }
    # Tuning on defined validation set of current iteration
    gs = GridSearch(model = ExtraTreesClassifier(), param_grid = param_grid, parallelize=False)
    gs.fit(X_train, y_train, X_val, y_val, scoring='f1')
    
    dump(gs.best_estimator_, './models/et_'+str(index)+'.joblib') 
    preds = gs.best_estimator_.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    
    for i,x in enumerate(test):
        if x in samples_classifications:
            samples_classifications[x]['ET']= preds[i]
        else:
            samples_classifications[x]= {'ET': preds[i]}
    print(confusion_matrix(y_test, preds))
    print('f1-score', gs.best_estimator_.score(X_test, y_test))
    print(gs.best_params)
    print('\n')

Iteration  0
[[109  74]
 [ 35 174]]
f1-score 0.7219387755102041
{'bootstrap': False, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 190}


Iteration  1
[[ 89  94]
 [ 47 162]]
f1-score 0.6403061224489796
{'bootstrap': False, 'max_depth': 50, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


Iteration  2
[[107  76]
 [ 34 175]]
f1-score 0.7193877551020408
{'bootstrap': False, 'max_depth': 32, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 170}


Iteration  3
[[ 97  86]
 [ 31 178]]
f1-score 0.701530612244898
{'bootstrap': False, 'max_depth': 32, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 190}


Iteration  4
[[107  76]
 [ 37 172]]
f1-score 0.7117346938775511
{'bootstrap': False, 'max_depth': 50, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 170}


Iteration  5
[[103  8

In [16]:
final_cm

array([[1031,  798],
       [ 400, 1688]])

RF tuning and cross validation

In [21]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])


for index, (train, test) in enumerate(skf.split(X_all, Y_all)):
    print('Iteration ', index)
    
    # Creating folds and validation set for tuning
    X_train = X_all[train]
    y_train = Y_all[train]
    X_test = X_all[test]
    y_test = Y_all[test]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    
    # Defining search space
    param_grid = {
                    'n_estimators': range(10,200,10),
                    'max_depth': [2,8,16,32,50],
                    'min_samples_split': [2,4,6],
                    'min_samples_leaf': [1,2],
                    'max_features': ['auto','sqrt','log2'],    
                    'bootstrap': [True, False]
    }
    # Tuning on defined validation set of current iteration
    gs = GridSearch(model = RandomForestClassifier(), param_grid = param_grid, parallelize=False)
    gs.fit(X_train, y_train, X_val, y_val)
    
    dump(gs.best_estimator_, './models/rf_'+str(index)+'.joblib') 
    preds = gs.best_estimator_.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    
    for i,x in enumerate(test):
        if x in samples_classifications:
            samples_classifications[x]['RF']= preds[i]
        else:
            samples_classifications[x]= {'RF': preds[i]}
    
    print(confusion_matrix(y_test, preds))
    print('f1-score', gs.best_estimator_.score(X_test, y_test))
    print(gs.best_params)
    print('\n')

Iteration  0
[[123  60]
 [ 38 171]]
f1-score 0.75
{'bootstrap': False, 'max_depth': 16, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 190}


Iteration  1
[[107  76]
 [ 55 154]]
f1-score 0.6658163265306123
{'bootstrap': False, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 180}


Iteration  2
[[114  69]
 [ 39 170]]
f1-score 0.7244897959183674
{'bootstrap': False, 'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 180}


Iteration  3
[[107  76]
 [ 34 175]]
f1-score 0.7193877551020408
{'bootstrap': False, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 160}


Iteration  4
[[119  64]
 [ 49 160]]
f1-score 0.7117346938775511
{'bootstrap': False, 'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}


Iteration  5
[[110  73]
 [ 44 165]

In [22]:
final_cm

array([[1128,  701],
       [ 459, 1629]])

SVM tuning and cross validation

In [31]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])


for index, (train, test) in enumerate(skf.split(X_all, Y_all)):
    print('Iteration ', index)
    
    # Creating folds and validation set for tuning
    X_train = X_all[train]
    y_train = Y_all[train]
    X_test = X_all[test]
    y_test = Y_all[test]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    
    # Defining search space
    param_grid = {
                    'C': [0.1, 1, 10, 100, 1000],
                    'kernel': ['poly', 'linear', 'rbf'],
                    'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                    'gamma': ['scale', 'auto']
    }
    # Tuning on defined validation set of current iteration
    gs = GridSearch(model = SVC(), param_grid = param_grid, parallelize=False)
    gs.fit(X_train, y_train, X_val, y_val)
    
    dump(gs.best_estimator_, './models/svc_'+str(index)+'.joblib') 
    preds = gs.best_estimator_.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    
    for i,x in enumerate(test):
        if x in samples_classifications:
            samples_classifications[x]['SVC']= preds[i]
        else:
            samples_classifications[x]= {'SVC': preds[i]}
    
    print(confusion_matrix(y_test, preds))
    print('f1-score', gs.best_estimator_.score(X_test, y_test))
    print(gs.best_params)
    print('\n')

Iteration  0
[[142  41]
 [ 44 165]]
f1-score 0.7831632653061225
{'C': 10, 'degree': 5, 'gamma': 'auto', 'kernel': 'poly'}


Iteration  1
[[128  55]
 [ 48 161]]
f1-score 0.7372448979591837
{'C': 10, 'degree': 5, 'gamma': 'scale', 'kernel': 'rbf'}


Iteration  2
[[125  58]
 [ 43 166]]
f1-score 0.7423469387755102
{'C': 10, 'degree': 7, 'gamma': 'auto', 'kernel': 'poly'}


Iteration  3
[[130  53]
 [ 52 157]]
f1-score 0.7321428571428571
{'C': 10, 'degree': 4, 'gamma': 'scale', 'kernel': 'poly'}


Iteration  4
[[143  40]
 [ 56 153]]
f1-score 0.7551020408163265
{'C': 10, 'degree': 5, 'gamma': 'auto', 'kernel': 'poly'}


Iteration  5
[[120  63]
 [ 52 157]]
f1-score 0.7066326530612245
{'C': 10, 'degree': 6, 'gamma': 'scale', 'kernel': 'poly'}


Iteration  6
[[119  64]
 [ 64 145]]
f1-score 0.673469387755102
{'C': 10, 'degree': 5, 'gamma': 'scale', 'kernel': 'poly'}


Iteration  7
[[127  55]
 [ 57 152]]
f1-score 0.7135549872122762
{'C': 10, 'degree': 8, 'gamma': 'auto', 'kernel': 'poly'}


Iterat

In [32]:
final_cm

array([[1285,  544],
       [ 517, 1571]])

Lightgbm tuning and cross validation

In [37]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])


for index, (train, test) in enumerate(skf.split(X_all, Y_all)):
    print('Iteration ', index)
    
    # Creating folds and validation set for tuning
    X_train = X_all[train]
    y_train = Y_all[train]
    X_test = X_all[test]
    y_test = Y_all[test]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    
    # Defining search space
    param_grid = {
                    'n_estimators': range(10,200,10),
                    'max_depth': [2,8,16,32,50],
                    'min_child_samples': [1,2],
                    'learning_rate': [0.01, 0.1, 0.2, 0.3]
    }
    # Tuning on defined validation set of current iteration
    gs = GridSearch(model = LGBMClassifier(), param_grid = param_grid, parallelize=False)
    gs.fit(X_train, y_train, X_val, y_val)
    
    dump(gs.best_estimator_, './models/lgbm_'+str(index)+'.joblib') 
    preds = gs.best_estimator_.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    
    for i,x in enumerate(test):
        if x in samples_classifications:
            samples_classifications[x]['LGBM']= preds[i]
        else:
            samples_classifications[x]= {'LGBM': preds[i]}
    
    print(confusion_matrix(y_test, preds))
    print('f1-score', gs.best_estimator_.score(X_test, y_test))
    print(gs.best_params)
    print('\n')

Iteration  0
[[134  49]
 [ 51 158]]
f1-score 0.7448979591836735
{'learning_rate': 0.3, 'max_depth': 50, 'min_child_samples': 2, 'n_estimators': 160}


Iteration  1
[[117  66]
 [ 63 146]]
f1-score 0.6709183673469388
{'learning_rate': 0.3, 'max_depth': 16, 'min_child_samples': 2, 'n_estimators': 140}


Iteration  2
[[123  60]
 [ 58 151]]
f1-score 0.6989795918367347
{'learning_rate': 0.3, 'max_depth': 32, 'min_child_samples': 1, 'n_estimators': 90}


Iteration  3
[[117  66]
 [ 55 154]]
f1-score 0.6913265306122449
{'learning_rate': 0.3, 'max_depth': 16, 'min_child_samples': 1, 'n_estimators': 190}


Iteration  4
[[124  59]
 [ 65 144]]
f1-score 0.6836734693877551
{'learning_rate': 0.2, 'max_depth': 50, 'min_child_samples': 1, 'n_estimators': 180}


Iteration  5
[[119  64]
 [ 56 153]]
f1-score 0.6938775510204082
{'learning_rate': 0.3, 'max_depth': 8, 'min_child_samples': 2, 'n_estimators': 190}


Iteration  6
[[ 97  86]
 [ 51 158]]
f1-score 0.6505102040816326
{'learning_rate': 0.2, 'max_dept

In [38]:
final_cm

array([[1185,  644],
       [ 582, 1506]])

ANN crossvalidation with tuning

In [50]:
def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [30]:
from contextlib import redirect_stdout

class Tuner(kt.BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', min_value = 8, max_value = 512, step = 8, default = 128)
        super().run_trial(trial, *args, **kwargs)

def create_hypermodel(hp):
    input = x = Input(shape = (47))

    for i in range(hp.Int('num_layers', min_value = 1, max_value = 5, step=2)):
        x = Dense(
            units = hp.Int('units_' + str(i), min_value = 32, max_value = 128, default = 120), activation = 'relu')(x)
        x = Dropout(
            rate = hp.Float('drop_rate_' + str(i), min_value = 0.0, max_value = 0.9, step = 0.1, default = 0.0)
        )(x)

    output = Dense( 1, 'sigmoid')(x)
    model = tf.keras.Model(input, output)

    lr = hp.Float('lr', min_value = 1e-5, max_value = 1e-1, default = 1e-3)
    decay = hp.Float('decay', min_value = 1e-6, max_value = 1e-2, default = 1e-3)
    model.compile(
        optimizer = tf.keras.optimizers.Adam(lr, decay = decay),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = [f1_score] #f1 score
    )
    return model


OBJECTIVE = kt.Objective('val_f1_score', direction = 'max')
MAX_TRIALS = 200
EXECUTIONS_PER_TRIAL = 1

callbacks = [
               tf.keras.callbacks.EarlyStopping(
                   monitor = 'val_f1_score',
                   min_delta = 0.00001,
                   patience = 5,
                   mode = 'max',
                   restore_best_weights = True
               )
  ]

hp = kt.HyperParameters()


skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm = np.array([[0, 0],[0, 0]])

for index, (train, test) in enumerate(skf.split(X_all, Y_all)):
    print('Iteration ', index)
    
    # Creating folds and validation set for tuning
    X_train = X_all[train]
    y_train = Y_all[train]
    X_test = X_all[test]
    y_test = Y_all[test]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    
    tuner = Tuner(
        create_hypermodel,
        objective = OBJECTIVE,
        hyperparameters = hp,
        tune_new_entries = True,
        max_trials = MAX_TRIALS,
        project_name='dnn2_'+str(index),
        executions_per_trial = EXECUTIONS_PER_TRIAL,
        seed = 42
    )
    
    tuner.search(
        X_train,
        y_train,
        epochs = 200,
        validation_data=(X_val, y_val),
        callbacks = callbacks,
        verbose = 1
    )
    best_model = tuner.get_best_models()[0]
    best_model.save('./models/dnn_'+str(index)+'.h5')
    preds = best_model.predict(X_test)
    y_pred = [ 1 if x>0.5 else 0 for x in preds]
    final_cm += confusion_matrix(y_test, y_pred)
    
    for i,x in enumerate(test):
        if x in samples_classifications:
            samples_classifications[x]['ANN']= y_pred[i]
        else:
            samples_classifications[x]= {'ANN': y_pred[i]}
    
    with open('tuning.txt', 'a') as f:
        f.write(str(confusion_matrix(y_test, y_pred)))
        with redirect_stdout(f):
            tuner.results_summary()
    print('\n')

Trial 200 Complete [00h 00m 07s]
val_f1_score: 0.7275139093399048

Best val_f1_score So Far: 0.7688758969306946
Total elapsed time: 00h 37m 22s
INFO:tensorflow:Oracle triggered exit




In [31]:
final_cm

array([[1029,  800],
       [ 484, 1604]])

In [32]:
with open('samples_classifications.pickle', 'wb') as handle:
    pickle.dump(samples_classifications, handle, protocol=pickle.HIGHEST_PROTOCOL)

<h3>Ensembling best classifiers</h3> (Subject independent)

In [20]:
dataset_covid_healthy = pd.read_pickle("dataset_covid_healthy.pkl")
_ = dataset_covid_healthy.pop('uuid')
Y = dataset_covid_healthy.pop('status')

In [ ]:

dataset_covid_healthy = StandardScaler().fit_transform(dataset_covid_healthy)

K-fold crossvalidation with best fold models

In [51]:
def build_nn():
    model = load_model('./models/dnn_4.h5', custom_objects={"f1_score" : f1score})
    model.compile(
        optimizer='Adam',
        loss='binary_crossentropy',
        metrics=[f1score]
    )
    return model

In [52]:
from sklearn.metrics import accuracy_score,f1_score, zero_one_loss

skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])

dataset_covid_healthy = pd.read_pickle("dataset_covid_healthy.pkl")
uuids_status_unique= dataset_covid_healthy.filter(items=['uuid','status']).drop_duplicates()
uuids = np.array(uuids_status_unique['uuid'])
y_uuids = np.array(uuids_status_unique['status'])

for train, test in skf.split(uuids, y_uuids):
    train_uuids = uuids[train]
    test_uuids = uuids[test]
    
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    for x in dataset_covid_healthy.iterrows():
        if x[1]['uuid'] in train_uuids:
            X_train.append(x[1][1:48])
            if x[1]['status']=='healthy':
                y_train.append(1)
            else:
                y_train.append(0)
        elif x[1]['uuid'] in test_uuids:
            X_test.append(x[1][1:48])
            if x[1]['status']=='healthy':
                y_test.append(1)
            else:
                y_test.append(0)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    #Normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    KNN_best = KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski', n_neighbors= 1, weights='uniform')
    SVC_best = SVC( C=10, degree= 5, gamma='auto', kernel='poly')
    SVC_best.probability=True
    LGBM_best = LGBMClassifier(learning_rate=0.3, max_depth=50, min_child_samples=2, n_estimators=160)
    ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)
    ANN_best._estimator_type = "classifier"

    voting_clf = VotingClassifier(estimators=[('KNN', KNN_best), ('SVC', SVC_best), ('LGBM', LGBM_best), ('ANN', ANN_best)], 
                                              voting='soft', flatten_transform=True, verbose=True, n_jobs=1)
    voting_clf.fit(X_train, y_train)
    preds =  voting_clf.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    print(confusion_matrix(y_test, preds))
    print('accuracy', accuracy_score(y_test, preds))
    print('f1_score', f1_score(y_test, preds))
    print('zero one loss', zero_one_loss(y_test, preds))

<ipython-input-52-ba74d13a0907>:48: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   3.8s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.3s
[Voting] ...................... (4 of 4) Processing ANN, total=  19.4s
[[72 55]
 [40 70]]
accuracy 0.5991561181434599
f1_score 0.5957446808510639
zero one loss 0.4008438818565401


<ipython-input-52-ba74d13a0907>:48: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   4.0s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.3s
[Voting] ...................... (4 of 4) Processing ANN, total=  18.4s
[[61 38]
 [46 65]]
accuracy 0.6
f1_score 0.6074766355140188
zero one loss 0.4


<ipython-input-52-ba74d13a0907>:48: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


...

[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   4.0s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.2s
[Voting] ...................... (4 of 4) Processing ANN, total=  19.5s
[[46 32]
 [36 51]]
accuracy 0.5878787878787879
f1_score 0.6
zero one loss 0.4121212121212121


<ipython-input-52-ba74d13a0907>:48: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   3.8s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.2s
[Voting] ...................... (4 of 4) Processing ANN, total=  18.6s
[[58 34]
 [33 61]]
accuracy 0.6397849462365591
f1_score 0.6455026455026456
zero one loss 0.3602150537634409


<ipython-input-52-ba74d13a0907>:48: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   3.8s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.2s
[Voting] ...................... (4 of 4) Processing ANN, total=  18.6s
[[47 32]
 [29 74]]
accuracy 0.6648351648351648
f1_score 0.7081339712918661
zero one loss 0.3351648351648352


In [53]:
final_cm

array([[1086,  743],
       [ 738, 1350]])

Leave one out crossvalidation with best fold models

In [54]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score,f1_score, zero_one_loss

loo = LeaveOneOut()

final_cm=np.array([[0, 0],[0, 0]])

dataset_covid_healthy = pd.read_pickle("dataset_covid_healthy.pkl")
uuids_status_unique= dataset_covid_healthy.filter(items=['uuid','status']).drop_duplicates()
uuids = np.array(uuids_status_unique['uuid'])
y_uuids = np.array(uuids_status_unique['status'])

for i, (train, test) in enumerate(loo.split(uuids)):
    train_uuids = uuids[train]
    test_uuids = uuids[test]
    
    #Loading best folds classifiers
    KNN_best = KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski', n_neighbors= 1, weights='uniform')
    SVC_best = SVC( C=10, degree= 5, gamma='auto', kernel='poly')
    SVC_best.probability=True
    LGBM_best = LGBMClassifier(learning_rate=0.3, max_depth=50, min_child_samples=2, n_estimators=160)
    ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)
    ANN_best._estimator_type = "classifier"
    
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    for x in dataset_covid_healthy.iterrows():
        if x[1]['uuid'] in train_uuids:
            X_train.append(x[1][1:48])
            if x[1]['status']=='healthy':
                y_train.append(1)
            else:
                y_train.append(0)
        elif x[1]['uuid'] in test_uuids:
            X_test.append(x[1][1:48])
            if x[1]['status']=='healthy':
                y_test.append(1)
            else:
                y_test.append(0)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    #Normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    
    voting_clf = VotingClassifier(estimators=[('KNN', KNN_best), ('SVC', SVC_best), ('LGBM', LGBM_best), ('ANN', ANN_best)], 
                                              voting='soft', flatten_transform=True, verbose=True)
    voting_clf.fit(X_train, y_train)
    preds =  voting_clf.predict(X_test)
    
    cm = np.array([[0, 0],[0, 0]])
    for pred in preds:
        if y_test[0] == pred:
            if pred == 1:
                cm += np.array([[0, 0],[0, 1]])
            else:
                cm += np.array([[1, 0],[0, 0]])
        else:
            if pred == 1:
                cm += np.array([[0, 1],[0, 0]])
            else:
                cm+= np.array([[0, 0],[1, 0]])
    final_cm += cm
    
    print('Iterazione', i, cm)
    print('accuracy', accuracy_score(y_test, preds))

<ipython-input-54-0786b9e49dc3>:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   4.5s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.2s
[Voting] ...................... (4 of 4) Processing ANN, total=  23.5s
Iterazione 0 [[0 0]
 [2 1]]
accuracy 0.3333333333333333


<ipython-input-54-0786b9e49dc3>:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   5.5s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.3s
[Voting] ...................... (4 of 4) Processing ANN, total=  29.6s
Iterazione 1 [[0 0]
 [1 3]]
accuracy 0.75


<ipython-input-54-0786b9e49dc3>:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


...

[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   4.7s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.3s
[Voting] ...................... (4 of 4) Processing ANN, total=  22.7s
Iterazione 867 [[0 0]
 [2 0]]
accuracy 0.0


<ipython-input-54-0786b9e49dc3>:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   6.6s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.6s
[Voting] ...................... (4 of 4) Processing ANN, total=  23.5s
Iterazione 868 [[0 0]
 [1 2]]
accuracy 0.6666666666666666


<ipython-input-54-0786b9e49dc3>:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  ANN_best = keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs= 200, verbose=False)


[Voting] ...................... (1 of 4) Processing KNN, total=   0.0s
[Voting] ...................... (2 of 4) Processing SVC, total=   5.1s
[Voting] ..................... (3 of 4) Processing LGBM, total=   0.2s
[Voting] ...................... (4 of 4) Processing ANN, total=  24.4s
Iterazione 869 [[0 0]
 [1 1]]
accuracy 0.5


In [55]:
final_cm

array([[1095,  734],
       [ 761, 1327]])